# Opening the box of gradient boosting decision trees (GBDT)

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

The aim of this notebook is 2-folds:

- dissecting a pure Python (unoptimized) implementation of GBRT
- use a profiler (here snakeviz) to identify the computational bottleneck.

We will explore the code in the file `tinygbt.py`.

TinyGBT (Tiny Gradient Boosted Trees) is a 200 line gradient boosted trees implementation written in pure Python.

First let's load some regression data.

In [1]:
import pandas as pd

print('Loading data...')
# load or create your dataset
df_train = pd.read_csv('datasets/regression.train', header=None, sep='\t')
df_test = pd.read_csv('datasets/regression.test', header=None, sep='\t')

df = pd.concat([df_train, df_test], axis=0)
y = df[0].values
X = df.drop(0, axis=1).values

Loading data...


Now we will split the data in train and test. Here the test data will be
used as **validation set** to do **"early stopping"** i.e. to stop adding trees
to the model if the validation loss increases for multiple successive boosting rounds.

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42)

#### Now let's run our TinyGBT code

In [3]:
from sklearn.metrics import mean_squared_error
from tinygbt import GBT

print('Start training...')
gbt = GBT(n_estimators=10, gamma=0., lambd=1,
          min_split_gain=0.1, max_depth=5,
          learning_rate=0.3)
gbt.fit(X_train, y_train, valid_set=(X_test, y_test), early_stopping_rounds=5)

print('Start predicting...')
y_pred = gbt.predict(X_test)
print('The MSE of prediction is:', mean_squared_error(y_test, y_pred))

Start training...
Training until validation scores don't improve for 5 rounds.
Iter   0, Train's L2: 0.3326061855, Valid's L2: 0.3329401346, Elapsed: 1.78 secs
Iter   1, Train's L2: 0.1925295668, Valid's L2: 0.2339036288, Elapsed: 1.77 secs
Iter   2, Train's L2: 0.1837464009, Valid's L2: 0.2295993444, Elapsed: 1.78 secs
Iter   3, Train's L2: 0.1815630268, Valid's L2: 0.2281974171, Elapsed: 1.78 secs
Iter   4, Train's L2: 0.1809400833, Valid's L2: 0.2278910397, Elapsed: 1.80 secs
Iter   5, Train's L2: 0.1807753271, Valid's L2: 0.2278302139, Elapsed: 1.81 secs
Iter   6, Train's L2: 0.1807266076, Valid's L2: 0.2278130572, Elapsed: 1.80 secs
Iter   7, Train's L2: 0.1807121121, Valid's L2: 0.2278076097, Elapsed: 1.83 secs
Iter   8, Train's L2: 0.1807077695, Valid's L2: 0.2278060130, Elapsed: 1.85 secs
Iter   9, Train's L2: 0.1807064624, Valid's L2: 0.2278055556, Elapsed: 1.84 secs
Training finished. Elapsed: 18.05 secs
Start predicting...
The MSE of prediction is: 0.2278055555875028


Let's first make sure this code gives comparable results with [lightgbm](https://lightgbm.readthedocs.io/en/latest/)

In [1]:
import lightgbm as lgb

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 1.,
    'bagging_fraction': 1.,
    'bagging_freq': 1,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
rmse_test = mean_squared_error(y_test, y_pred)
print(f'The MSE of prediction is: {rmse_test}')

ModuleNotFoundError: No module named 'lightgbm'

And also with scikit-learn [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html)

In [3]:
# from sklearn.experimental import enable_hist_gradient_boosting  # uncomment this for sklearn < 1.0
from sklearn.ensemble import HistGradientBoostingRegressor
est = HistGradientBoostingRegressor(
    max_iter=10,
    validation_fraction=0.2,
    random_state=42,
    l2_regularization=0.,
    min_samples_leaf=20,
    learning_rate=0.3,
    n_iter_no_change=5).fit(X_train, y_train)
y_pred = est.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred)
print(f'The MSE of prediction is: {rmse_test}')

NameError: name 'X_train' is not defined

### Let's do a round of profiling using snakeviz

While lightgbm is implemented in C++ and HistGradientBoostingRegressor with Cython, our TinyGBT is implemented in pure Python. The consequence is that our code is much slower. To identify the computational bottleneck let's profile our code. For this we will use snakeviz.

There are many other profilers such as:
- line_profiler: https://github.com/pyutils/line_profiler (easy line by line profiling for Python code)
- viztracer: https://viztracer.readthedocs.io (allows to profile low level code including compiled Cython code)

In [2]:
%load_ext snakeviz

gbt = GBT(n_estimators=2,  # do only two rounds of boosting
          gamma=0., lambd=1,
          min_split_gain=0.1, max_depth=5,
          learning_rate=0.3)

%snakeviz gbt.fit(X_train, y_train, valid_set=(X_test, y_test), early_stopping_rounds=5)

ModuleNotFoundError: No module named 'snakeviz'

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Where is most of the time spent?</li>
         <li>What is the complexity of the <code>TreeNode.build</code> method as a function of the number of samples and the number of features?</li>
         <li>What could we do about this?</li>
     </ul>
</div>